In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

with zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/train_crop.zip') as z:
  z.extractall('')
# name: cropped_large

In [ ]:
import csv
import numpy as np
import os
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import cv2
import pandas as pd
import csv
from PIL import Image

In [ ]:
train_csv = '/content/drive/MyDrive/Colab Notebooks/train.csv'
category_csv = '/content/drive/MyDrive/Colab Notebooks/category.csv'
train_data_path = '/content/cropped_large'

label_to_index = {}
index_to_label = {}

# Read the CSV file and populate the dictionaries
with open(category_csv, 'r', newline='') as file:
  reader = csv.reader(file)
  for idx, row in enumerate(reader):
    if idx > 0:
      index, label = row[0], row[1]
      label_to_index[label] = int(index)
      index_to_label[index] = label


Mapping from name to number:
{'Art Carney': 0, 'Akemi Darenogare': 1, 'Billy Bob Thornton': 2, 'Bill Daley': 3, 'Amanda Bynes': 4, 'Anna Friel': 5, 'Alexander Skarsgard': 6, 'Anna Faris': 7, 'Bill Pullman': 8, 'Anjelica Huston': 9, 'Angela Bassett': 10, 'Alx James': 11, 'Benjamin Bratt': 12, 'Andy Garcia': 13, 'Bernie Mac': 14, 'Andreea Diaconu': 15, 'Anna Kendrick': 16, 'Anna Sui': 17, 'Barry Pepper': 18, 'Alexis Thorpe': 19, 'Ashley Judd': 20, 'Amy Ryan': 21, 'Analeigh Tipton': 22, 'Anne Hathaway': 23, 'Amanda Peet': 24, 'Amanda Crew': 25, 'Adriana Barraza': 26, 'Ashton Kutcher': 27, 'Ali Larter': 28, 'Arnold Schwarzenegger': 29, 'Amy Adams': 30, 'Berenice Bejo': 31, 'Alessandra Ambrosio': 32, 'Audrey Hepburn': 33, 'Andy Samberg': 34, 'Abigail Breslin': 35, 'Aaron Judge': 36, 'Andrew Luck': 37, 'Bill Murray': 38, 'Anna Paquin': 39, 'Angelina Jolie': 40, 'Barbra Streisand': 41, 'Ashley Graham': 42, 'Anthony Mackie': 43, 'Alec Baldwin': 44, 'Andy Murray': 45, 'Armin Mueller-Stahl': 46,

In [ ]:
import pandas as pd

train_data = pd.read_csv(train_csv)
print(train_data.head())

   Unnamed: 0 File Name       Category
0           0     0.jpg  Audrey Tautou
1           1     1.jpg   Adam Sandler
2           2     2.jpg    Anna Paquin
3           3     3.jpg    Ava Gardner
4           4     4.jpg      Amy Adams


In [ ]:
batch_size = 128
epochs = 10
workers = 0 if os.name == 'nt' else 10

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [ ]:
class Mydataset(Dataset):
  def __init__(self, csv_file, images_dir, index_map, transform=None):
    self.images_dir = images_dir
    self.transform = transform
    self.index_map = index_map # filename to name
    label_frame = pd.read_csv(csv_file)

    # Create a dictionary to map filenames to labels
    self.label_dict = pd.Series(label_frame.Category.values,
                      index=label_frame['File Name']).to_dict()

    # List all jpg files
    self.filenames = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]

  def __len__(self):
    return len(self.filenames)

  def __getitem__(self, idx):
    filename = self.filenames[idx]
    img_path = os.path.join(self.images_dir, filename)
    image = Image.open(img_path).convert("RGB")

    label = self.index_map[self.label_dict[filename]]

    if self.transform:
      image = self.transform(image)

    return image, label

# transform
train_transform= transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.ToTensor(),
  torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

train_dataset = Mydataset(csv_file= train_csv, images_dir=train_data_path,
                      index_map=label_to_index, transform=train_transform)

In [ ]:
image_data = np.arange(len(train_dataset))
train_data = image_data[:int(0.8 * len(image_data))] # 80% of data used for train
valid_data = image_data[int(0.8 * len(image_data)):] # 20% of data used for validation

random_indices = torch.randperm(len(train_data)) # change the order of training data

train_loader = DataLoader(
    train_dataset,
    num_workers=workers,
    batch_size=batch_size,
    shuffle=True,
)

val_loader = DataLoader(
    valid_data,
    num_workers=workers,
    batch_size=batch_size,
    shuffle=True,
)

In [ ]:
import torchvision.models as models

# Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

for param in model.parameters():
  param.requires_grad = False

model.fc = nn.Linear(2048, len(label_to_index))

model.to(device)

# Print the model
print(model)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 178MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Load the ResNet50 model
model = torchvision.models.resnet50(pretrained=True)
model.to(device)

model.train()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
        epoch + 1, epochs, i + 1, len(train_loader), loss.item()))

    torch.save(model.state_dict(), f'resnet50_{epoch}.pt')

# Save the trained model
torch.save(model.state_dict(), 'resnet50.pt')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Step [100/1883], Loss: 4.6849
Epoch [1/10], Step [200/1883], Loss: 4.1362
Epoch [1/10], Step [300/1883], Loss: 4.0134
Epoch [1/10], Step [400/1883], Loss: 3.8495
Epoch [1/10], Step [500/1883], Loss: 4.2940
Epoch [1/10], Step [600/1883], Loss: 3.4222
Epoch [1/10], Step [700/1883], Loss: 3.2066
Epoch [1/10], Step [800/1883], Loss: 3.1992
Epoch [1/10], Step [900/1883], Loss: 3.2499
Epoch [1/10], Step [1000/1883], Loss: 3.2690
Epoch [1/10], Step [1100/1883], Loss: 3.2561
Epoch [1/10], Step [1200/1883], Loss: 2.9907
Epoch [1/10], Step [1300/1883], Loss: 2.4930
Epoch [1/10], Step [1400/1883], Loss: 2.5583
Epoch [1/10], Step [1500/1883], Loss: 3.0505
Epoch [1/10], Step [1600/1883], Loss: 2.4767
Epoch [1/10], Step [1700/1883], Loss: 2.7149
Epoch [1/10], Step [1800/1883], Loss: 2.5378
Epoch [2/10], Step [100/1883], Loss: 3.0152
Epoch [2/10], Step [200/1883], Loss: 2.2747
Epoch [2/10], Step [300/1883], Loss: 2.2248
Epoch [2/10], Step [400/1883], Loss: 1.5622
Epoch [2/10], Step [500

In [ ]:
class TestDataset(Dataset):
  def __init__(self, images_dir, transform=None):
    self.images_dir = images_dir
    self.transform = transform
    self.filenames = []

    # List all jpg files
    self.filenames = [f for f in os.listdir(test_data_path) if f.endswith('.jpg')]

  def __len__(self):
    return len(self.filenames)

  def __getitem__(self, idx):
    filename = self.filenames[idx]
    img_path = os.path.join(self.images_dir, filename)
    image = Image.open(img_path).convert("RGB")
    # Seperate num.jpg to num and jpg
    num, _ = os.path.splitext(filename)
    label = int(num)

    return image, label

test_data_path = '/content/drive/MyDrive/Colab Notebooks/test_crop_jo'

test_dataset = TestDataset(images_dir=test_data_path, transform=train_transform)

test_loader = DataLoader(
    test_dataset,
    num_workers=workers,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
# Load the trained model
model.load_state_dict(torch.load('resnet50.pt'))
model.to(device)

model.eval()

correct = 0
total = 0
with torch.no_grad():
  for images, labels in val_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the model : %d %%' % (100 * correct / total))

In [ ]:
# Load the trained model
model.load_state_dict(torch.load('resnet50_8.pt'))
model.to(device)
model.eval()

predictions = []
for filename in os.listdir(test_data_path):
  if filename.endswith('.jpg'):
    # Load the image
    img_path = os.path.join(test_data_path, filename)
    image = Image.open(img_path).convert("RGB")
    print(image)

    image = train_transform(image)
    image = image.to(device)
    output = model(image.unsqueeze(0))
    _, predicted = torch.max(output.data, 1)
    predicted_label = index_to_label.get(predicted.item())
    predictions.append(predicted_label)


<PIL.Image.Image image mode=RGB size=736x535 at 0x7C125E3AE0E0>
<PIL.Image.Image image mode=RGB size=300x300 at 0x7C125E3AF550>
<PIL.Image.Image image mode=RGB size=454x455 at 0x7C125E3AE410>
<PIL.Image.Image image mode=RGB size=750x1063 at 0x7C125E3AF2B0>
<PIL.Image.Image image mode=RGB size=634x417 at 0x7C125E3AD4B0>
<PIL.Image.Image image mode=RGB size=240x320 at 0x7C125E3AFE20>
<PIL.Image.Image image mode=RGB size=3417x2563 at 0x7C125E3ADDB0>
<PIL.Image.Image image mode=RGB size=1080x1920 at 0x7C125E3AD4B0>
<PIL.Image.Image image mode=RGB size=483x750 at 0x7C125E3AD150>
<PIL.Image.Image image mode=RGB size=377x198 at 0x7C125E3AC4F0>
<PIL.Image.Image image mode=RGB size=615x409 at 0x7C125E3AC790>
<PIL.Image.Image image mode=RGB size=540x377 at 0x7C125E3AE0E0>
<PIL.Image.Image image mode=RGB size=900x600 at 0x7C125E3AD4B0>
<PIL.Image.Image image mode=RGB size=306x441 at 0x7C125E3AC790>
<PIL.Image.Image image mode=RGB size=1280x720 at 0x7C125E3AE9E0>
<PIL.Image.Image image mode=RGB si

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


<PIL.Image.Image image mode=RGB size=1218x1920 at 0x7C125E3AF550>
<PIL.Image.Image image mode=RGB size=450x450 at 0x7C125E3747F0>
<PIL.Image.Image image mode=RGB size=900x664 at 0x7C125E3AF550>
<PIL.Image.Image image mode=RGB size=866x1390 at 0x7C125E3AD510>
<PIL.Image.Image image mode=RGB size=590x250 at 0x7C125E3ADDB0>
<PIL.Image.Image image mode=RGB size=683x1024 at 0x7C125E3AD510>
<PIL.Image.Image image mode=RGB size=432x594 at 0x7C125E3AF550>
<PIL.Image.Image image mode=RGB size=634x792 at 0x7C13955DA470>
<PIL.Image.Image image mode=RGB size=1200x800 at 0x7C125E3ADDB0>
<PIL.Image.Image image mode=RGB size=400x400 at 0x7C125E3ADDB0>
<PIL.Image.Image image mode=RGB size=1377x742 at 0x7C13955DA470>
<PIL.Image.Image image mode=RGB size=600x450 at 0x7C13955DA470>
<PIL.Image.Image image mode=RGB size=300x190 at 0x7C125E3ADDB0>
<PIL.Image.Image image mode=RGB size=600x756 at 0x7C125E3747F0>
<PIL.Image.Image image mode=RGB size=1024x752 at 0x7C125E3ADDB0>
<PIL.Image.Image image mode=RGB s

In [1]:
# The submission csv
'''
file_name = 'submission_lee4461.csv'

with open(file_name, mode='w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(['Id', 'Category']) # the header

  num = 0
  for pred in predictions:
    writer.writerow([num, pred])
    num += 1
'''
print('CSV file is created')

CSV file is created
